# Finding Association for Store Dataset
- Support of `1.5%`
- Confidence of `40%`
- For $T(X)$ be number of transactions containing X
## Support
$$
    Support(\{X\}\rightarrow \{Y\}) = \frac{T(X,Y)}{T(All)}
$$
## Confidence
$$
    Confidence(\{X\}\rightarrow \{Y\}) = \frac{T(X,Y)}{T(X)}
$$

In [1]:
"""
Data Mining Assignment 2 [Date: 11 Feb 21]
"""
__author__='Black D Chase'
__version__='0.3.3'

In [2]:
# Imports

x=!pip show tqdm
if 'WARNING' in x[0]:
    !pip install tqdm
from tqdm import tqdm
#For loading bar, to check if it's not stuck. For smaller values of `connectivityCount` and bigger values of n in nConnectivityCount.

x=!pip show IPython
if 'WARNING' in x[0]:
    !pip install IPython
from IPython import display
#For showing Result Image

In [3]:
store = open('testAppriori.csv')
pairConnectivity=[]
pairConnectivity.append({})
associations = {}

supportPer = 0.4
# Support of 40%

minimumSupport = 2
# Number of minimum Transaction for it to be considered (Number of trxn*supportPer)

confidencePer = 0.6
# Confidence of 40%

# Change these values to get different result according to the need

In [4]:
def nConnectivityCal(n,minimumSupport=2,debugMode=False):
    global pairConnectivity
    if n<=0:
        print(n,"very small")
        return 
    if len(pairConnectivity)<n:
        nConnectivityCal(n-1,minimumSupport)
    if len(pairConnectivity)==n:
        pairConnectivity.append({})
    if n==1:
        for i in pairConnectivity[0].keys():
            for j in pairConnectivity[0][i]:
                k = frozenset([j])
                if k not in pairConnectivity[1].keys():
                    pairConnectivity[1][k]=set()
                else:
                    pairConnectivity[1][k].add(i)
    else:
        keys = list(pairConnectivity[n-1].keys())
        kRange = []
        for i in range(len(keys)-1):
            for j in range(i+1,len(keys)):
                kRange.append([i,j])
        for i,j in tqdm(kRange):
            intersect = keys[i].intersection(keys[j])
            diff = keys[i].symmetric_difference(keys[j])
            k=frozenset(intersect.union(diff))
            if len(intersect)==n-2 and len(diff)==2:
                a = set(pairConnectivity[n-1][keys[i]])
                b = set(pairConnectivity[n-1][keys[j]])
                chkPocket = a.intersection(b)
                if len(chkPocket)>=minimumSupport:
                    pairConnectivity[n][k] = set(chkPocket)
    return pairConnectivity[n]

In [5]:
def findSupport(x,y,debugMode=False):
    global minimumSupport, supportPer,pairConnectivity
    x = frozenset(x)
    y = frozenset(y)
    z = x.union(y)
    if len(z)>len(pairConnectivity)-1:
        nConnectivityCal(len(z),minimumSupport)
    try:
        totalTransactions = len(pairConnectivity[0].keys())
        transactionXY = len(pairConnectivity[len(z)][z])
    except:
        if debugMode:
            print("No such transaction")
    # if zero - could be 0,1,2.. (less than min connectivity)
    try:
        supp=transactionXY/totalTransactions
    except:
        supp=0
        if debugMode:
            print("Total Number of transaction is 0")
    if debugMode:
        print("Support: ",supp*100,"%, where required was ",supportPer*100,"%")
        print("Acceptable") if (supp>=supportPer) else print("Not acceptable")
    return supp

In [6]:
def findConfidence(x,y,debugMode=False):
    global minimumSupport,confidencePer,pairConnectivity
    x = frozenset(x)
    y = frozenset(y)
    z = x.union(y)
    if len(z)>len(pairConnectivity)-1:
        nConnectivityCal(len(z),minimumSupport)
    try:
        transactionX = len(pairConnectivity[len(x)][x])
        transactionXY = len(pairConnectivity[len(z)][z])
    except:
        if debugMode:
            print("No such transaction")
    # if zero - could be 0,1,2.. (less than min connectivity)
    try:
        conf = transactionXY/transactionX
    except:
        conf=0
        if debugMode:
            print("Number of transaction containing ",x," is less then ",minimumSupport)
    if debugMode:
        print("Confidence: ",conf*100,"%, where required was ",confidencePer*100,"%")
        print("Acceptable") if (conf>=confidencePer) else print("Not acceptable")
    return conf

In [7]:
def updateAssociation(x,y,debugMode=False):
    global associations,supportPer,confidencePer
    a,b = set(x),set(y)
    temp = str(a) + " -> " + str(b)
    supp = findSupport(x,y)
    conf = findConfidence(x,y)
    if supp!=0:
        assoMsg = " Support= "+str(supp)+" Confidence= "+str(conf)
        if supp>=supportPer:
            assoMsg+="\n Support Acceptable"
        else:
            assoMsg+="\n Support Unacceptable"
        if conf>=confidencePer:
            assoMsg+=", Confidence Acceptable"
        else:
            assoMsg+=", Confidence Unacceptable"
        if debugMode:
            print(temp,":",assoMsg)
        associations[temp]=assoMsg
        return True
    return False

In [8]:
# Making ID's for Transactions
t=0
for i in store.readlines():
    x = i.rsplit(',')
    x[-1]=x[-1][:-1]
    # Removing '\n'
    x=set(x)
    if '' in x:
        x.remove('')
    x=frozenset(x)
    pairConnectivity[0]["TX"+str(t)]=x
    t+=1

- `pairConnectivity[0]` will save all transations with respect to thier Transation ID we just made.
- `pairConnectivity[n]` will have a dictonary, in which `Items` Is the Item set  with `n` distinguished items. This set will be mapped to set of transaction which have this item set.
`frozenset(Items)-> set(transaction(Items))`

In [9]:
nConnectivityCal(4,minimumSupport)

100%|██████████| 3/3 [00:00<00:00, 28859.89it/s]
0it [00:00, ?it/s]


{}

In [10]:
pairConnectivity[3]

{frozenset({'I1', 'I2', 'I3'}): {'TX7', 'TX8'}}

## For all the association rules in this Dataset:

In [11]:
keys_1 = list(pairConnectivity[1].keys())
keys_2 = list(pairConnectivity[2].keys())
keys_3 = list(pairConnectivity[3].keys())
# Checking Upto 3 levels

# 2-2
temp = []
for i in range(len(keys_2)):
    for j in range(i+1,len(keys_2)):
        temp.append([i,j])
for i,j in tqdm(temp):
    x,y = keys_2[i],keys_2[j]
    updateAssociation(x,y)
    updateAssociation(y,x)

# 1-1
temp = []
for i in range(len(keys_1)):
    for j in range(i+1,len(keys_1)):
        temp.append([i,j])
for i,j in tqdm(temp):
    x,y = keys_1[i],keys_1[j]
    updateAssociation(x,y)
    updateAssociation(y,x)

# 1-2
temp = []
for i in range(len(keys_2)):
    for j in range(len(keys_1)):
        temp.append([i,j])
for i,j in tqdm(temp):
    x,y = keys_2[i],keys_1[j]
    updateAssociation(y,x)
    updateAssociation(x,y)

# 3-3
temp = []
for i in range(len(keys_3)):
    for j in range(i+1,len(keys_3)):
        temp.append([i,j])
for i,j in tqdm(temp):
    x,y = keys_3[i],keys_3[j]
    updateAssociation(x,y)
    updateAssociation(y,x)

# 1-3
temp = []
for i in range(len(keys_1)):
    for j in range(len(keys_3)):
        temp.append([i,j])
for i,j in tqdm(temp):
    x,y = keys_1[i],keys_3[j]
    updateAssociation(x,y)
    updateAssociation(y,x)

# 2-3
temp = []
for i in range(len(keys_2)):
    for j in range(len(keys_3)):
        temp.append([i,j])
for i,j in tqdm(temp):
    x,y = keys_2[i],keys_3[j]
    updateAssociation(y,x)
    updateAssociation(x,y)


100%|██████████| 15/15 [00:00<00:00, 63167.23it/s]
0it [00:00, ?it/s]
100%|██████████| 3/3 [00:00<00:00, 12748.64it/s]


In [12]:
associations

{"{'I2', 'I1'} -> {'I2', 'I3'}": ' Support= 0.2222222222222222 Confidence= 0.6666666666666666\n Support Unacceptable, Confidence Acceptable',
 "{'I2', 'I3'} -> {'I2', 'I1'}": ' Support= 0.2222222222222222 Confidence= 0.6666666666666666\n Support Unacceptable, Confidence Acceptable',
 "{'I2', 'I1'} -> {'I1', 'I3'}": ' Support= 0.2222222222222222 Confidence= 0.6666666666666666\n Support Unacceptable, Confidence Acceptable',
 "{'I1', 'I3'} -> {'I2', 'I1'}": ' Support= 0.2222222222222222 Confidence= 0.5\n Support Unacceptable, Confidence Unacceptable',
 "{'I2', 'I3'} -> {'I1', 'I3'}": ' Support= 0.2222222222222222 Confidence= 0.6666666666666666\n Support Unacceptable, Confidence Acceptable',
 "{'I1', 'I3'} -> {'I2', 'I3'}": ' Support= 0.2222222222222222 Confidence= 0.5\n Support Unacceptable, Confidence Unacceptable',
 "{'I2'} -> {'I1'}": ' Support= 0.3333333333333333 Confidence= 0.5\n Support Unacceptable, Confidence Unacceptable',
 "{'I1'} -> {'I2'}": ' Support= 0.3333333333333333 Confid